In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import numpy as np
import pandas as pd

def load_ratings():
    COL_NAME = ['uid','mid','rating','timestamp']
    df = pd.read_csv('./dataset/ml-1m/ratings.dat',sep='::', header=None, engine='python', names=COL_NAME)
    return df

def load_movies():
    COL_NAME = ['mid','movie_name','movie_genre']
    df = pd.read_csv('./dataset/ml-1m/movies.dat',sep='::', header=None, engine='python', names=COL_NAME)
    return df

def load_users():
    COL_NAME = ['uid','user_fea1','user_fea2','user_fea3','user_fea4']
    df = pd.read_csv('./dataset/ml-1m/users.dat',sep='::', header=None, engine='python', names=COL_NAME)
    return df

def text2seq(text, n_genre):
    tokenizer = Tokenizer(lower=True, split='|',filters='', num_words=n_genre)
    tokenizer.fit_on_texts(text)
    seq = tokenizer.texts_to_sequences(text)
    seq = pad_sequences(seq, maxlen=3,padding='post')
    return seq

def _model_fm_part(inputs, params):
    
    n_uid = params['n_uid']
    n_mid = params['n_mid']
    n_genre = params['n_genre']
    k = params['k']
    
    input_uid, input_mid = inputs
    
    emb_uid = Embedding(n_uid + 1, output_dim=k, input_length=1, name = 'fm_emb_uid')(input_uid)
    emb_uid = Flatten()(emb_uid)
    emb_mid = Embedding(n_mid + 1, output_dim = k, input_length=1, name = 'fm_emb_mid')(input_mid)
    emb_mid = Flatten()(emb_mid)
    
    return Multiply(name = 'fm_elementwise_mul')([emb_uid,emb_mid])

def _model_mlp_part(inputs, params):
        
    input_uid, input_mid = inputs
    mlp_layers = params['mlp_layers']
    n_uid = params['n_uid']
    n_mid = params['n_mid']
    n_genre = params['n_genre']
    k = params['k']
    emb_uid = Embedding(n_uid + 1, k, input_length=1, name = 'mlp_emb_uid')(input_uid)
    emb_uid = Flatten()(emb_uid)
    emb_mid = Embedding(n_mid + 1, k, input_length=1, name = 'mlp_emb_mid')(input_mid)
    emb_mid = Flatten()(emb_mid)
    
    output = concatenate([emb_uid, emb_mid])
    for h in mlp_layers:
        output = Dense(h, activation='relu')(output)
        
    return output

def df2xy(ratings):
    x = [ratings.user_fea3.values, 
         ratings.uid.values, 
         ratings.mid.values, 
         np.concatenate(ratings.movie_genre.values).reshape(-1,3)]
    y = ratings.rating.values
    return x,y

def deep_fm_model(n_uid, n_mid, n_genre, k, dnn_dim, dnn_dr):
    # numerica features
    fea4_input = Input((1,), name = 'input_fea4')
    num_inputs = [fea4_input]
    # single level categorical features
    uid_input = Input((1,), name = 'input_uid')
    mid_input = Input((1,), name= 'input_mid')
    cat_sl_inputs = [uid_input, mid_input]

    # multi level categorical features (with 3 genres at most)
    genre_input = Input((3,), name = 'input_genre')
    cat_ml_inputs = [genre_input]

    inputs = num_inputs + cat_sl_inputs + cat_ml_inputs

    # first order fm
    # all tensors are reshape to (None, 1)
    num_dense_1d = [Dense(1, name = 'num_dense_1d_fea4')(fea4_input)]
    cat_sl_embed_1d = [Embedding(n_uid + 1, 1, name = 'cat_embed_1d_uid')(uid_input),
                        Embedding(n_mid + 1, 1, name = 'cat_embed_1d_mid')(mid_input)]
    cat_ml_embed_1d = [Embedding(n_genre + 1, 1, name = 'cat_embed_1d_genre')(genre_input)]

    cat_sl_embed_1d = [Reshape((1,))(i) for i in cat_sl_embed_1d]
    cat_ml_embed_1d = [Flatten()(i) for i in cat_ml_embed_1d]

    # add all tensors
    y_fm_1d = Add(name = 'fm_1d_output')(num_dense_1d + cat_sl_embed_1d + cat_ml_embed_1d)


    # second order fm
    # reshape all tensors to (None, k)
    num_dense_2d = [Dense(k, name = 'num_dense_2d_fea4')(fea4_input)]
    cat_sl_embed_2d = [Embedding(n_uid + 1, k, name = 'cat_embed_2d_uid')(uid_input), 
                       Embedding(n_mid + 1, k, name = 'cat_embed_2d_mid')(mid_input)]
    cat_ml_embed_2d = [Embedding(n_genre + 1, k, name = 'cat_embed_2d_genre')(genre_input)]
    cat_ml_embed_2d = [Lambda(lambda x: K.mean(x, axis=1), name = 'embed_2d_mean')(i) for i in cat_ml_embed_2d]

    num_dense_2d = [Reshape((1,k))(i) for i in num_dense_2d]
    cat_ml_embed_2d = [Reshape((1,k))(i) for i in cat_ml_embed_2d]

    embed_2d = Concatenate(axis=1, name = 'concat_embed_2d')(num_dense_2d + cat_sl_embed_2d + cat_ml_embed_2d)

    # calcuate the interactions by simplication
    # sum of (x1*x2) = sum of (0.5*[(xi)^2 - (xi^2)])
    tensor_sum = Lambda(lambda x: K.sum(x, axis = 1), name = 'sum_of_tensors')
    tensor_square = Lambda(lambda x: K.square(x), name = 'square_of_tensors')

    sum_of_embed = tensor_sum(embed_2d)
    square_of_embed = tensor_square(embed_2d)

    square_of_sum = Multiply()([sum_of_embed, sum_of_embed])
    sum_of_square = tensor_sum(square_of_embed)

    sub = Subtract()([square_of_sum, sum_of_square])
    sub = Lambda(lambda x: x*0.5)(sub)
    y_fm_2d = Reshape((1,), name = 'fm_2d_output')(tensor_sum(sub))


    # dnn part
    y_dnn = Flatten(name = 'flat_embed_2d')(embed_2d)
    for h in dnn_dim:
        y_dnn = Dropout(dnn_dr)(y_dnn)
        y_dnn = Dense(h, activation='relu')(y_dnn)
    y_dnn = Dense(1, activation='relu', name = 'deep_output')(y_dnn)

    # combinded deep and fm parts
    y = Concatenate()([y_fm_1d, y_fm_2d, y_dnn])
    y = Dense(1, name = 'deepfm_output')(y)

    fm_model_1d = Model(inputs, y_fm_1d)
    fm_model_2d = Model(inputs, y_fm_2d)
    deep_model = Model(inputs, y_dnn)
    deep_fm_model = Model(inputs, y)
    
    return fm_model_1d, fm_model_2d, deep_model, deep_fm_model

## Load Data

In [2]:
n_genre = 15

ratings = load_ratings()
movies = load_movies()
users = load_users()

print(ratings.head())
print(movies.head())

movies['movie_genre'] = text2seq(movies.movie_genre.values, n_genre=n_genre).tolist()
print(movies.head())

ratings = ratings.join(movies.set_index('mid'), on = 'mid', how = 'left')
ratings = ratings.join(users.set_index('uid'), on = 'uid', how = 'left')
(ratings.head())

   uid   mid  rating  timestamp
0    1  1193       5  978300760
1    1   661       3  978302109
2    1   914       3  978301968
3    1  3408       4  978300275
4    1  2355       5  978824291
   mid                          movie_name                   movie_genre
0    1                    Toy Story (1995)   Animation|Children's|Comedy
1    2                      Jumanji (1995)  Adventure|Children's|Fantasy
2    3             Grumpier Old Men (1995)                Comedy|Romance
3    4            Waiting to Exhale (1995)                  Comedy|Drama
4    5  Father of the Bride Part II (1995)                        Comedy
   mid                          movie_name movie_genre
0    1                    Toy Story (1995)   [9, 2, 0]
1    2                      Jumanji (1995)   [7, 9, 0]
2    3             Grumpier Old Men (1995)   [2, 5, 0]
3    4            Waiting to Exhale (1995)   [2, 1, 0]
4    5  Father of the Bride Part II (1995)   [2, 0, 0]


,uid,mid,rating,timestamp,movie_name,movie_genre,user_fea1,user_fea2,user_fea3,user_fea4
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),"[1, 0, 0]",F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),"[9, 13, 0]",F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),"[13, 5, 0]",F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),"[1, 0, 0]",F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)","[9, 2, 0]",F,1,10,48067


## Split Data

In [3]:
in_train_flag = np.random.random(len(ratings)) <= 0.9
train_data = ratings.loc[in_train_flag,]
valid_data = ratings.loc[~in_train_flag,]
train_x, train_y = df2xy(train_data)
valid_x, valid_y = df2xy(valid_data)

## Load Model

In [4]:
params = {
    'n_uid': ratings.uid.max(),
    'n_mid': ratings.mid.max(),
    'n_genre': 14,
    'k':20,
    'dnn_dim':[64,64],
    'dnn_dr': 0.5
}

fm_model_1d, fm_model_2d, deep_model, deep_fm_model = deep_fm_model(**params)

In [ ]:
# train  model
deep_fm_model.compile(loss = 'MSE', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=3)
callbacks = [early_stop]
deep_fm_model.fit(train_x, train_y, epochs=30, batch_size=2048, validation_split=0.1, callbacks = callbacks)

/home/liuchr/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 810005 samples, validate on 90001 samples
Epoch 1/30
810005/810005 [==============================] - 43s 54us/step - loss: 8.4493 - val_loss: 5.4553
Epoch 2/30
810005/810005 [==============================] - 41s 51us/step - loss: 1.4727 - val_loss: 4.4670
Epoch 3/30
810005/810005 [==============================] - 41s 51us/step - loss: 0.9880 - val_loss: 4.2425
Epoch 4/30
810005/810005 [==============================] - 41s 51us/step - loss: 0.9314 - val_loss: 4.1805
Epoch 5/30
810005/810005 [==============================] - 41s 50us/step - loss: 0.9130 - val_loss: 4.0764
Epoch 6/30
810005/810005 [==============================] - 41s 51us/step - loss: 0.9009 - val_loss: 3.9853
Epoch 7/30
810005/810005 [==============================] - 7s 8us/step - loss: 0.8910 - val_loss: 3.8919
Epoch 8/30
810005/810005 [==============================] - 6s 7us/step - loss: 0.8811 - val_loss: 3.7791
Epoch 9/30
810005/810005 [==============================] - 6s 7us/step - loss: 0.8712 - 

## Model Architect

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(fm_model_1d, to_file='./image/fm_model_1d.png',show_shapes=True, show_layer_names=True)
plot_model(fm_model_2d, to_file='./image/fm_model_2d.png',show_shapes=True, show_layer_names=True)
plot_model(deep_model, to_file='./image/deep_model.png',show_shapes=True, show_layer_names=True)
plot_model(deep_fm_model, to_file='./image/deep_fm_model.png',show_shapes=True, show_layer_names=True)

## 1st order FM model
![title](./image/fm_model_1d.png)

## 2nd order FM model
![title](./image/fm_model_2d.png)

## Deep model
![title](./image/deep_model.png)

## Deep FM model
![title](./image/deep_fm_model.png)